In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython import display
import pymongo
import random
import numpy as np
from bokeh.io import output_notebook
from climatools.cliradlw.cliradlw_latest import *
from climatools.html.html import *
from climatools.spectrum import *
output_notebook()
pd.set_option('max_colwidth', -1)

Loading BokehJS ...

In [45]:
LINKS.fillna('', inplace=True)
LINKS.index.names = ['absorber(s)']
LINKS.sort_index(axis=1, level=1, inplace=True)
LINKS.sort_index(axis=0, inplace=True)
display.display(display.HTML(LINKS.to_html(escape=False)))

lblnew band(s),[3b],[5]
cliradlw band(s),[4],[7]
absorber(s),,
{'co2': 0.0008},,saw
"{'h2o': 'atmpro', 'co2': 0.0008}",saw mls trp,


In [3]:
CLIENT = pymongo.MongoClient(host='localhost', port=27017)

In [4]:
LINKS = pd.DataFrame(columns=pd.MultiIndex(levels=[[], []], labels=[[], []], names=['lblnew band(s)', 'cliradlw band(s)']))
LINKS.index.name = ['absorber(s)']

# 2 x $CO_2$

In [5]:
MOL = 'co2'
CONC = 2 * 400e-6
COMMITNUMBER = '1013f91'
NV, DV = 1000, .001
ATMPRO = dict(mls=294, saw=257, trp=300)

## Atmosphere compositions

We first set the concentration of CO2 to that double of the usual 4e-6 ppmv.  Then we split the overall composition into those where CO2 overlaps with other gases and those where only CO2 is present.

In [6]:
atmcomp = AtmComposition.cliradlw_nongreys()
atmcomp.gasconcs['co2'] = CONC
atmcomp.gasinbands = {b:gs for b, gs in atmcomp.gasinbands.items() if MOL in gs}

In [7]:
atmcomps_olp = [AtmComposition(gasconcs=atmcomp.gasconcs, gasinbands={b: gs}) 
                for b, gs in atmcomp.gasinbands.items() if len(gs) > 1]

In [8]:
atmcomps_bft = [AtmComposition(gasconcs=atmcomp.gasconcs, gasinbands={b:[MOL]}) for b, _ in atmcomp.gasinbands.items()]

In [9]:
## CO2 only case

In [10]:
atmpro = 'saw'

In [11]:
atmbft = atmcomps_bft[4]

In [12]:
parm_cli = atmbft.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])

In [13]:
data_cli = parm_cli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)

In [14]:
parms_lbl = atmbft.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro], nv=NV, dv=DV)

In [15]:
parms_lbl[0].avail_commits(collection=CLIENT.lblnew.bestfit_lw)

['a22ab94']

In [16]:
data_lbl = parms_lbl[0].modeldata_pymongo(collection=CLIENT.lblnew.bestfit_lw)

In [17]:
ida = show_results(dlbl=data_lbl, dcli=data_cli, add_idanchor=True)
show_href(df=LINKS, idanchor=ida, dlbl=data_lbl, dcli=data_cli)

-----------------------------

<a id="05225883484"></a>

[7] {'co2': 0.0008} saw

Run parameters                          \
                                CLIRAD                  LBLNEW   
atmpro                 saw              saw                      
band                   [7]              5                        
commitnumber           1013f91          a22ab94                  
conc                   -                0.0008                   
dv                     -                0.001                    
klin                   -                6.5e-24                  
molecule               {'co2': 0.0008}  co2                      
ng_adju                -                [0, 0]                   
ng_refs                -                [1, 2]                   
nv                     -                1000                     
option_compute_btable  -                0                        
option_compute_ktable  -                0                        
option_wgt_flux        -                1                        
option_wgt_k           -                1                        
ref_pts                -                [[1, 250], [500, 250]]   
tsfc                   257              257                      
vmax                   -                1100                     
vmin                   -                980                      
w_diffuse              -                [[1.75], [1.66, 1.9]]    
wgt                    -                [[0.75], [0.75, 0.95]]   

                                Fit parameters  
                                      (co2, 5)  
atmpro                 mls                      
band                   5                        
commitnumber           a06b618                  
conc                   0.0004                   
dv                     0.001                    
klin                   6.5e-24                  
molecule               co2                      
ng_adju                [0, 0]                   
ng_refs                [1, 2]                   
nv                     1000                     
option_compute_btable  0                        
option_compute_ktable  1                        
option_wgt_flux        1                        
option_wgt_k           1                        
ref_pts                [(1, 250), (500, 250)]   
tsfc                   294                      
vmax                   1100                     
vmin                   980                      
w_diffuse              [(1.75,), (1.66, 1.9)]   
wgt                    [(0.75,), (0.75, 0.95)]

--------------------------

In [177]:
# CO2 overlapped with other gases

In [36]:
atmpro = 'trp'

In [37]:
atmolp = atmcomps_olp[1]

In [38]:
parm_cli = atmolp.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])

In [39]:
data_cli = parm_cli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)

In [40]:
parms_lbl = atmolp.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro], dv=DV, nv=NV)

In [41]:
parms_lbl

[<class 'climatools.parameters.LBLnewOverlapParam'>
 atmpro: trp
 band: 3b
 commitnumber: None
 dv: 0.001
 molecule: {'h2o': 'atmpro', 'co2': 0.0008}
 nv: 1000
 tsfc: 300]

In [42]:
parms_lbl[0].avail_commits(collection=CLIENT.lblnew.overlap_lw)

['a22ab94']

In [43]:
data_lbl = parms_lbl[0].modeldata_pymongo(collection=CLIENT.lblnew.overlap_lw)

In [44]:
ida = show_results(dlbl=data_lbl, dcli=data_cli, add_idanchor=True)
show_href(idanchor=ida, df=LINKS, dlbl=data_lbl, dcli=data_cli)

-----------------------------

<a id="81860855655"></a>

[4] {'h2o': 'atmpro', 'co2': 0.0008} trp

Run parameters  \
                                                 CLIRAD   
atmpro                 trp                                
band                   [4]                                
commitnumber           1013f91                            
conc                   -                                  
dv                     -                                  
klin                   -                                  
molecule               {'h2o': 'atmpro', 'co2': 0.0008}   
ng_adju                -                                  
ng_refs                -                                  
nv                     -                                  
option_compute_btable  -                                  
option_compute_ktable  -                                  
option_wgt_flux        -                                  
option_wgt_k           -                                  
ref_pts                -                                  
tsfc                   300                                
vmax                   -                                  
vmin                   -                                  
w_diffuse              -                                  
wgt                    -                                  

                                                         \
                                                 LBLNEW   
atmpro                 trp                                
band                   3b                                 
commitnumber           a22ab94                            
conc                   -                                  
dv                     0.001                              
klin                   -                                  
molecule               {'co2': 0.0008, 'h2o': 'atmpro'}   
ng_adju                -                                  
ng_refs                -                                  
nv                     1000                               
option_compute_btable  -                                  
option_compute_ktable  -                                  
option_wgt_flux        -                                  
option_wgt_k           -                                  
ref_pts                -                                  
tsfc                   300                                
vmax                   -                                  
vmin                   -                                  
w_diffuse              -                                  
wgt                    -                                  

                                                                   Fit parameters  \
                                                                        (co2, 3b)   
atmpro                 mls                                                          
band                   3b                                                           
commitnumber           a22ab94                                                      
conc                   0.0004                                                       
dv                     0.001                                                        
klin                   0                                                            
molecule               co2                                                          
ng_adju                [0, 0]                                                       
ng_refs                [5, 4]                                                       
nv                     1000                                                         
option_compute_btable  1                                                            
option_compute_ktable  1                                                            
option_wgt_flux        1                                                            
option_wgt_k           1                                                            
ref_pts                [(1, 250), (10, 250)]                                       

--------------------------

In [35]:
ida = show_results(dlbl=data_lbl, dcli=data_cli, add_idanchor=True)
show_href(idanchor=ida, df=LINKS, dlbl=data_lbl, dcli=data_cli)

-----------------------------

<a id="00270408353"></a>

[4] {'h2o': 'atmpro', 'co2': 0.0008} mls

Run parameters  \
                                                 CLIRAD   
atmpro                 mls                                
band                   [4]                                
commitnumber           1013f91                            
conc                   -                                  
dv                     -                                  
klin                   -                                  
molecule               {'h2o': 'atmpro', 'co2': 0.0008}   
ng_adju                -                                  
ng_refs                -                                  
nv                     -                                  
option_compute_btable  -                                  
option_compute_ktable  -                                  
option_wgt_flux        -                                  
option_wgt_k           -                                  
ref_pts                -                                  
tsfc                   294                                
vmax                   -                                  
vmin                   -                                  
w_diffuse              -                                  
wgt                    -                                  

                                                         \
                                                 LBLNEW   
atmpro                 mls                                
band                   3b                                 
commitnumber           a22ab94                            
conc                   -                                  
dv                     0.001                              
klin                   -                                  
molecule               {'co2': 0.0008, 'h2o': 'atmpro'}   
ng_adju                -                                  
ng_refs                -                                  
nv                     1000                               
option_compute_btable  -                                  
option_compute_ktable  -                                  
option_wgt_flux        -                                  
option_wgt_k           -                                  
ref_pts                -                                  
tsfc                   294                                
vmax                   -                                  
vmin                   -                                  
w_diffuse              -                                  
wgt                    -                                  

                                                                   Fit parameters  \
                                                                        (co2, 3b)   
atmpro                 mls                                                          
band                   3b                                                           
commitnumber           a22ab94                                                      
conc                   0.0004                                                       
dv                     0.001                                                        
klin                   0                                                            
molecule               co2                                                          
ng_adju                [0, 0]                                                       
ng_refs                [5, 4]                                                       
nv                     1000                                                         
option_compute_btable  1                                                            
option_compute_ktable  1                                                            
option_wgt_flux        1                                                            
option_wgt_k           1                                                            
ref_pts                [(1, 250), (10, 250)]                                       

--------------------------

In [387]:
ida = show_results(dlbl=data_lbl, dcli=data_cli, add_idanchor=True)
show_href(idanchor=ida, df=LINKS, dlbl=data_lbl, dcli=data_cli)

-----------------------------

<a id="60122515550"></a>

[4] {'h2o': 'atmpro', 'co2': 0.0008} saw

Run parameters  \
                                                 CLIRAD   
atmpro                 saw                                
band                   [4]                                
commitnumber           1013f91                            
conc                   -                                  
dv                     -                                  
klin                   -                                  
molecule               {'h2o': 'atmpro', 'co2': 0.0008}   
ng_adju                -                                  
ng_refs                -                                  
nv                     -                                  
option_compute_btable  -                                  
option_compute_ktable  -                                  
option_wgt_flux        -                                  
option_wgt_k           -                                  
ref_pts                -                                  
tsfc                   257                                
vmax                   -                                  
vmin                   -                                  
w_diffuse              -                                  
wgt                    -                                  

                                                         \
                                                 LBLNEW   
atmpro                 saw                                
band                   3b                                 
commitnumber           a22ab94                            
conc                   -                                  
dv                     0.001                              
klin                   -                                  
molecule               {'co2': 0.0008, 'h2o': 'atmpro'}   
ng_adju                -                                  
ng_refs                -                                  
nv                     1000                               
option_compute_btable  -                                  
option_compute_ktable  -                                  
option_wgt_flux        -                                  
option_wgt_k           -                                  
ref_pts                -                                  
tsfc                   257                                
vmax                   -                                  
vmin                   -                                  
w_diffuse              -                                  
wgt                    -                                  

                                                                   Fit parameters  \
                                                                        (co2, 3b)   
atmpro                 mls                                                          
band                   3b                                                           
commitnumber           a22ab94                                                      
conc                   0.0004                                                       
dv                     0.001                                                        
klin                   0                                                            
molecule               co2                                                          
ng_adju                [0, 0]                                                       
ng_refs                [5, 4]                                                       
nv                     1000                                                         
option_compute_btable  1                                                            
option_compute_ktable  1                                                            
option_wgt_flux        1                                                            
option_wgt_k           1                                                            
ref_pts                [(1, 250), (10, 250)]                                       

--------------------------

# Fin

In [187]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
